# Intelligent Tic Tac Toe Agent using Reinforcement Learning

In [1]:
import numpy as np

In [2]:
LENGTH = 3

### Intelligent Agent Class 

In [3]:
class Agent():
    
    def __init__(self, eps=0.1, alpha=0.5):
        self.eps = eps
        self.alpha = alpha
        self.verbose = False
        self.state_history = []
    
    def set_V(self, V):
        self.V = V
    
    def set_symbol(self, sym):
        self.sym = sym
    
    def set_verbose(self, verbose):
        self.verbose = verbose
    
    def reset_history(self):
        self.state_history = []
    
    def take_action(self, env):
        r = np.random.randn()
        best_state = None
        
        if r < self.eps:
            if self.verbose:
                print('Taking a random action')
            possible_moves = []
            for i in range(LENGTH):
                for j in range(LENGTH):
                    if env.is_empty(i, j):
                        possible_moves.append((i, j))
            idx = np.random.choice(len(possible_moves))
            next_move = possible_moves[idx]
        else:
            pos2value = {}
            next_move = None
            best_value = -1
            for i in range(LENGTH):
                for j in range(LENGTH):
                    if env.is_empty(i, j):
                        env.board[i, j] = self.sym
                        state = env.get_state()
                        env.board[i, j] = 0
                        pos2value[(i, j)] = self.V[state]
                        if self.V[state] > best_value:
                            best_value = self.V[state]
                            best_state = state
                            next_move = (i, j)
                            
            if self.verbose:
                print('Taking a greedy action')
                for i in range(LENGTH):
                    print('------------------')
                    for j in range(LENGTH):
                        if env.is_empty(i, j):
                            print(' %.2f|' % pos2value[(i,j)], end='')
                        else:
                            print('  ', end='')
                            if env.board[i,j] == env.x:
                                print('x  |', end='')
                            elif env.board[i,j] == env.o:
                                print('o  |', end='')
                            else:
                                print('   |', end='')
                    print('')
                print('------------------')
        
        env.board[next_move[0], next_move[1]] = self.sym
    
    def update_state_history(self, state):
        self.state_history.append(state)
    
    def update(self, env):
        reward = env.reward(self.sym)
        target = reward
        for prev in reversed(self.state_history):
            value = self.V[prev] + self.alpha * (target - self.V[prev])
            self.V[prev] = value
            target = value
        self.reset_history()

### Environment Class in which Agent trains

In [4]:
class Environment():
    
    def __init__(self):
        self.board = np.zeros((LENGTH, LENGTH))
        self.x = -1
        self.o = 1
        self.winner = None
        self.ended = False
        self.num_states = 3 ** (LENGTH * LENGTH)
    
    def is_empty(self, i, j):
        return self.board[i, j] == 0
    
    def reward(self, sym):
        if not self.game_over():
            return 0
        return 1 if self.winner == sym else 0
    
    def get_state(self):
        k = 0
        h = 0
        for i in range(LENGTH):
            for j in range(LENGTH):
                if self.board[i, j] == 0:
                    v = 0
                elif self.board[i, j] == self.x:
                    v = 1
                elif self.board[i, j] == self.o:
                    v = 2
                h += (3 ** k) * v
                k += 1
        return h
    
    def game_over(self, force_recalculate=False):
        if not force_recalculate and self.ended:
            return self.ended
        
        for i in range(LENGTH):
            for player in (self.x, self.o):
                if self.board[i].sum() == player * LENGTH:
                    self.winner = player
                    self.ended = True
                    return True
                
        for j in range(LENGTH):
            for player in (self.x, self.o):
                if self.board[:, j].sum() == player * LENGTH:
                    self.winner = player
                    self.ended = True
                    return True
                
        for player in (self.x, self.o):
            if self.board.trace() == player * LENGTH:
                self.winner = player
                self.ended = True
                return True
            
            if np.fliplr(self.board).trace() == player * LENGTH:
                self.winner = player
                self.ended = True
                return True
        
        if np.all((self.board == 0) == False):
            self.winner = None
            self.ended = True
            return True
        
        self.winner = None
        return False
    
    def draw_board(self):
        for i in range(LENGTH):
            print('-------------')
            for j in range(LENGTH):
                print('  ', end='')
                if self.board[i,j] == self.x:
                    print('x ', end='')
                elif self.board[i,j] == self.o:
                    print('o ', end='')
                else:
                    print('  ', end='')
            print('')
        print('-------------')

### Class for a human player against the trained Agent

In [5]:
class Human():
    
    def __init__(self):
        pass
    
    def set_symbol(self, sym):
        self.sym = sym
    
    def take_action(self, env):
        while True:
            move = input('Enter coordinates i,j for your next move (i,j=0..2): ')
            i, j = move.split(',')
            i = int(i)
            j = int(j)
            if env.is_empty(i, j):
                env.board[i, j] = self.sym
                break
    
    def update(self, env):
        pass
    
    def update_state_history(self, state):
        pass

#### Initialize state values as follows:
- if x wins, V(s) = 1
- if x loses or draw, V(s) = 0
- otherwise, V(s) = 0.5

In [6]:
def initialV_x(env, state_winner_triples):
    V = np.zeros(env.num_states)
    for state, winner, ended in state_winner_triples:
        if ended:
            if winner == env.x:
                v = 1
            else:
                v = 0
        else:
            v = 0.5
        V[state] = v
    return V

- This is (almost) the opposite of initial V for player x
- Since everywhere where x wins (1), o loses (0)
- But a draw is still 0 for o

In [7]:
def initialV_o(env, state_winner_triples):
    V = np.zeros(env.num_states)
    for state, winner, ended in state_winner_triples:
        if ended:
            if winner == env.o:
                v = 1
            else:
                v = 0
        else:
            v = 0.5
        V[state] = v
    return V

- Recursive function that will return all possible states (as ints) and who the corresponding winner is for those states (if any)
- (i, j) refers to the next cell on the board to permute (we need to try -1, 0, 1)
- Impossible games are ignored, i.e. 3x's and 3o's in a row simultaneously since that will never happen in a real game

In [8]:
def get_state_hash_and_winner(env, i=0, j=0):
    results = []

    for v in (0, env.x, env.o):
        env.board[i,j] = v
        if j == 2:
            if i == 2:
                state = env.get_state()
                ended = env.game_over(force_recalculate=True)
                winner = env.winner
                results.append((state, winner, ended))
            else:
                results += get_state_hash_and_winner(env, i + 1, 0)
        else:
            results += get_state_hash_and_winner(env, i, j + 1)

    return results

#### Game starting function

In [9]:
def play_game(p1, p2, env, draw=False):
    current_player = None
    
    while not env.game_over():
        if current_player == p1:
            current_player = p2
        else:
            current_player = p1
        
        if draw:
            if draw == 1 and current_player == p1:
                env.draw_board()
            if draw == 2 and current_player == p2:
                env.draw_board()
        
        current_player.take_action(env)
        
        state = env.get_state()
        p1.update_state_history(state)
        p2.update_state_history(state)
        
    if draw:
        env.draw_board()
    
    p1.update(env)
    p2.update(env)

In [10]:
p1 = Agent()
p2 = Agent()
env = Environment()

* Remove comments to see the agents getting trained.

In [11]:
state_winner_triples = get_state_hash_and_winner(env)
Vx = initialV_x(env, state_winner_triples)
p1.set_V(Vx)
Vo = initialV_o(env, state_winner_triples)
p2.set_V(Vo)

p1.set_symbol(env.x)
p2.set_symbol(env.o)

# p1.set_verbose(True)
# p2.set_verbose(True)

In [12]:
T = 10000
for t in range(T):
    if t % 200 == 0:
        print(t)
    play_game(p1, p2, Environment())

0
200
400
600
800
1000
1200
1400
1600
1800
2000
2200
2400
2600
2800
3000
3200
3400
3600
3800
4000
4200
4400
4600
4800
5000
5200
5400
5600
5800
6000
6200
6400
6600
6800
7000
7200
7400
7600
7800
8000
8200
8400
8600
8800
9000
9200
9400
9600
9800


- Agents are trained
- Play with any one of them

In [13]:
human = Human()
human.set_symbol(env.o)

In [14]:
while(True):
    p1.set_verbose(True)
    play_game(p1, human, Environment(), draw=2)
    answer = input('Play again? [Y/n]: ')
    if answer and answer.lower()[0] == 'n':
        break

Taking a random action
-------------
          x 
-------------
            
-------------
            
-------------
Enter coordinates i,j for your next move (i,j=0..2): 2,2
Taking a greedy action
------------------
 0.69| 0.73|  x  |
------------------
 0.70| 0.66| 0.43|
------------------
 0.56| 0.60|  o  |
------------------
-------------
      x   x 
-------------
            
-------------
          o 
-------------
Enter coordinates i,j for your next move (i,j=0..2): 0,0
Taking a random action
-------------
  o   x   x 
-------------
      x     
-------------
          o 
-------------
Enter coordinates i,j for your next move (i,j=0..2): 2,1
Taking a greedy action
------------------
  o  |  x  |  x  |
------------------
 0.04|  x  | 0.04|
------------------
 1.00|  o  |  o  |
------------------
-------------
  o   x   x 
-------------
      x     
-------------
  x   o   o 
-------------
Play again? [Y/n]: 1,0
Taking a greedy action
------------------
 0.59| 0.47| 0.68|
-------